In [1]:
import copy
import re
import pandas as pd
import numpy as np
from experiments import ExperimentModule
from experiments import (
    save_plots, 
    write_row, 
    write_opcounts,
    save_circuit,
    create_dir,
    get_random_state
)
import matplotlib.pyplot as plt
from itertools import product

%matplotlib inline

In [2]:
results_dir = "results/runs_shell_w_entanglement/random_states"
run_range = list(range(1, 11))
#qubit_range = [3, 4, 5, 6, 7, 8]
qubit_range = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#qubit_range = [3, 4, 5, 6, 7, 8, 9, 10]

# EPS values are mapped separately
#eps_values = [0.5, 0.1, 0.05, 0.01, 0.005, 0.001]
eps_values = [0.5, 0.1, 0.05, 0.01]

### Random State type

In [3]:
state_type = "random_state"

In [4]:
#[0.5, 0.1, 0.05, 0.01, 0.005, 0.001]
original_results_dict = {
    "num_qubits": [],
    0.5: [],
    0.1: [],
    0.05: [],
    0.01: [],
}

modified_results_dict = copy.deepcopy(original_results_dict)

for (r_dir, st_type, num_qubits, eps) in product([results_dir], [state_type], qubit_range, eps_values):
    if not num_qubits in original_results_dict["num_qubits"]: 
        original_results_dict["num_qubits"] += [num_qubits]
        modified_results_dict["num_qubits"] += [num_qubits]

    original_results_dict[eps] += [[]]
    modified_results_dict[eps] += [[]]

    for run_idx in run_range:
        fid_loss_original = np.loadtxt(f"{results_dir}/{st_type}/run_{run_idx}/csv/original_fidloss_{num_qubits}qb_{eps}eps.csv", delimiter=",")
        data_original = fid_loss_original[-1][-1] if len(fid_loss_original.shape) > 1 else fid_loss_original[-1]
        original_results_dict[eps][-1] += [data_original.round(4)]

        fid_loss_modified = np.loadtxt(f"{results_dir}/{st_type}/run_{run_idx}/csv/modified_fidloss_{num_qubits}qb_{eps}eps.csv", delimiter=",")
        data_modified = fid_loss_modified[-1][-1] if len(fid_loss_modified.shape) > 1 else fid_loss_modified[-1]
        modified_results_dict[eps][-1]+= [data_modified.round(4)]

In [5]:
default_dict = {
    "num_qubits": qubit_range,
    0.5: [],
    0.1: [],
    0.05: [],
    0.01: [],
}
mean_original_results_dict = copy.deepcopy(default_dict)
mean_modified_results_dict = copy.deepcopy(default_dict)

stds_original_results_dict = copy.deepcopy(default_dict)
stds_modified_results_dict = copy.deepcopy(default_dict)

for eps in eps_values:
    mean_original_results_dict[eps] = np.mean(original_results_dict[eps], axis=1)
    stds_original_results_dict[eps] = np.std(original_results_dict[eps], axis=1).round(2)

    mean_modified_results_dict[eps] = np.mean(modified_results_dict[eps], axis=1)
    stds_modified_results_dict[eps] = np.std(modified_results_dict[eps], axis=1).round(2)

Original fidloss

In [6]:
df_mean_original = pd.DataFrame(mean_original_results_dict)
df_mean_original = df_mean_original.set_index("num_qubits")
display(df_mean_original)

df_stds_original = pd.DataFrame(stds_original_results_dict)
df_stds_original = df_stds_original.set_index("num_qubits")
df_stds_original

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.16070,0.16070,0.00006,0.00138
4,0.21840,0.21840,0.17120,0.00085
5,0.21760,0.21760,0.19290,0.15679
6,0.22370,0.22370,0.22150,0.19552
7,0.23930,0.23930,0.23020,0.21580
8,0.23250,0.23596,0.73335,0.22139
9,0.25170,0.25170,0.80360,0.62207
10,0.83675,0.81631,0.67229,0.90795
11,0.47313,0.55462,0.52192,0.70257


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00
6,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00
8,0.00,0.01,0.04,0.00
9,0.00,0.00,0.03,0.04
10,0.07,0.03,0.05,0.02
11,0.09,0.23,0.07,0.03


In [7]:
print(df_mean_original.to_latex(float_format="%.4f"))

\begin{tabular}{lrrrr}
\toprule
 & 0.500000 & 0.100000 & 0.050000 & 0.010000 \\
num_qubits &  &  &  &  \\
\midrule
3 & 0.1607 & 0.1607 & 0.0001 & 0.0014 \\
4 & 0.2184 & 0.2184 & 0.1712 & 0.0009 \\
5 & 0.2176 & 0.2176 & 0.1929 & 0.1568 \\
6 & 0.2237 & 0.2237 & 0.2215 & 0.1955 \\
7 & 0.2393 & 0.2393 & 0.2302 & 0.2158 \\
8 & 0.2325 & 0.2360 & 0.7334 & 0.2214 \\
9 & 0.2517 & 0.2517 & 0.8036 & 0.6221 \\
10 & 0.8367 & 0.8163 & 0.6723 & 0.9079 \\
11 & 0.4731 & 0.5546 & 0.5219 & 0.7026 \\
12 & 0.9166 & 0.9338 & 0.6443 & 0.9104 \\
13 & 0.7782 & 0.8255 & 0.6370 & 0.9485 \\
14 & 0.9864 & 0.9871 & 0.9095 & 0.9681 \\
15 & 0.9187 & 0.9006 & 0.9594 & 0.9966 \\
\bottomrule
\end{tabular}



Modified fidloss

In [8]:
df_mean_modified = pd.DataFrame(mean_modified_results_dict)
df_mean_modified = df_mean_modified.set_index("num_qubits")
display(df_mean_modified)

df_stds_modified = pd.DataFrame(stds_modified_results_dict)
df_stds_modified = df_stds_modified.set_index("num_qubits")
df_stds_modified

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.05950,0.05950,0.01456,0.00393
4,0.17420,0.17420,0.12411,0.00222
5,0.18270,0.18270,0.71420,0.13970
6,0.21140,0.21140,0.56381,0.71242
7,0.24200,0.24200,0.60720,0.21420
8,0.23310,0.23310,0.22610,0.93235
9,0.25090,0.25090,0.93960,0.62754
10,0.89502,0.87953,0.45987,0.99052
11,0.89677,0.89364,0.95651,0.25108


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.00,0.00,0.03,0.01
4,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00
6,0.00,0.00,0.04,0.00
7,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.02
9,0.00,0.00,0.01,0.05
10,0.02,0.02,0.08,0.00
11,0.02,0.01,0.01,0.01


In [9]:
print(df_mean_modified.to_latex(float_format="%.4f"))

\begin{tabular}{lrrrr}
\toprule
 & 0.500000 & 0.100000 & 0.050000 & 0.010000 \\
num_qubits &  &  &  &  \\
\midrule
3 & 0.0595 & 0.0595 & 0.0146 & 0.0039 \\
4 & 0.1742 & 0.1742 & 0.1241 & 0.0022 \\
5 & 0.1827 & 0.1827 & 0.7142 & 0.1397 \\
6 & 0.2114 & 0.2114 & 0.5638 & 0.7124 \\
7 & 0.2420 & 0.2420 & 0.6072 & 0.2142 \\
8 & 0.2331 & 0.2331 & 0.2261 & 0.9323 \\
9 & 0.2509 & 0.2509 & 0.9396 & 0.6275 \\
10 & 0.8950 & 0.8795 & 0.4599 & 0.9905 \\
11 & 0.8968 & 0.8936 & 0.9565 & 0.2511 \\
12 & 0.8654 & 0.8620 & 0.9363 & 0.9824 \\
13 & 0.9408 & 0.9481 & 0.9732 & 0.9765 \\
14 & 0.9940 & 0.9935 & 0.7897 & 0.9930 \\
15 & 0.9894 & 0.9867 & 0.9995 & 0.9621 \\
\bottomrule
\end{tabular}



In [10]:
df_mean_original < df_mean_modified


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,False,False,True,True
4,False,False,False,True
5,False,False,True,False
6,False,False,True,True
7,True,True,True,False
8,True,False,False,True
9,False,False,True,True
10,True,True,False,True
11,True,True,True,False


### Random Sparse States

In [11]:
state_type = "random_sparse_state"
density = 0.3

In [12]:
#[0.5, 0.1, 0.05, 0.01, 0.005, 0.001]
original_results_dict = {
    "num_qubits": [],
    0.5: [],
    0.1: [],
    0.05: [],
    0.01: [],
}

modified_results_dict = copy.deepcopy(original_results_dict)

for (r_dir, st_type, num_qubits, eps) in product([results_dir], [state_type], qubit_range, eps_values):
    if not num_qubits in original_results_dict["num_qubits"]: 
        original_results_dict["num_qubits"] += [num_qubits]
        modified_results_dict["num_qubits"] += [num_qubits]

    original_results_dict[eps] += [[]]
    modified_results_dict[eps] += [[]]

    for run_idx in run_range:
        fid_loss_original = np.loadtxt(f"{results_dir}/{st_type}/run_{run_idx}/density_{density}/csv/original_fidloss_{num_qubits}qb_{eps}eps.csv", delimiter=",")
        data_original = fid_loss_original[-1][-1] if len(fid_loss_original.shape) > 1 else fid_loss_original[-1]
        original_results_dict[eps][-1] += [data_original.round(4)]

        fid_loss_modified = np.loadtxt(f"{results_dir}/{st_type}/run_{run_idx}/density_{density}/csv/modified_fidloss_{num_qubits}qb_{eps}eps.csv", delimiter=",")
        data_modified = fid_loss_modified[-1][-1] if len(fid_loss_modified.shape) > 1 else fid_loss_modified[-1]
        modified_results_dict[eps][-1]+= [data_modified.round(4)]

In [13]:
default_dict = {
    "num_qubits": qubit_range,
    0.5: [],
    0.1: [],
    0.05: [],
    0.01: [],
}
mean_original_results_dict = copy.deepcopy(default_dict)
mean_modified_results_dict = copy.deepcopy(default_dict)

stds_original_results_dict = copy.deepcopy(default_dict)
stds_modified_results_dict = copy.deepcopy(default_dict)

for eps in eps_values:
    mean_original_results_dict[eps] = np.mean(original_results_dict[eps], axis=1)
    stds_original_results_dict[eps] = np.std(original_results_dict[eps], axis=1).round(2)

    mean_modified_results_dict[eps] = np.mean(modified_results_dict[eps], axis=1)
    stds_modified_results_dict[eps] = np.std(modified_results_dict[eps], axis=1).round(2)

In [14]:
df_mean_original = pd.DataFrame(mean_original_results_dict)
df_mean_original = df_mean_original.set_index("num_qubits")
display(df_mean_original)
display(df_mean_modified)

df_stds_original = pd.DataFrame(stds_original_results_dict)
df_stds_original = df_stds_original.set_index("num_qubits")
#df_stds_original

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.90650,0.90650,0.47247,0.54325
4,0.88969,0.91338,0.13790,0.00511
5,0.50068,0.50064,0.40350,0.32201
6,0.59459,0.61325,0.51499,0.40651
7,0.77317,0.74214,0.69640,0.68881
8,0.74250,0.74250,0.75178,0.70597
9,0.76120,0.76120,0.99352,0.92595
10,0.90657,0.90311,0.76806,0.95512
11,0.81337,0.83254,0.91915,0.84499


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.05950,0.05950,0.01456,0.00393
4,0.17420,0.17420,0.12411,0.00222
5,0.18270,0.18270,0.71420,0.13970
6,0.21140,0.21140,0.56381,0.71242
7,0.24200,0.24200,0.60720,0.21420
8,0.23310,0.23310,0.22610,0.93235
9,0.25090,0.25090,0.93960,0.62754
10,0.89502,0.87953,0.45987,0.99052
11,0.89677,0.89364,0.95651,0.25108


In [15]:
display(df_stds_original)
display(df_stds_modified)

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.00,0.00,0.20,0.24
4,0.13,0.06,0.00,0.01
5,0.00,0.00,0.00,0.02
6,0.00,0.06,0.01,0.00
7,0.08,0.06,0.00,0.02
8,0.00,0.00,0.02,0.00
9,0.00,0.00,0.00,0.02
10,0.01,0.01,0.00,0.00
11,0.02,0.06,0.01,0.01


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.00,0.00,0.03,0.01
4,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00
6,0.00,0.00,0.04,0.00
7,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.02
9,0.00,0.00,0.01,0.05
10,0.02,0.02,0.08,0.00
11,0.02,0.01,0.01,0.01


In [16]:
print("--"*100)
print("Original Mean")
print("--"*100)
print(df_mean_original.to_latex(float_format="%.4f"))

print("--"*100)
print("Modified Mean")
print("--"*100)
print(df_mean_modified.to_latex(float_format="%.4f"))

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Original Mean
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
\begin{tabular}{lrrrr}
\toprule
 & 0.500000 & 0.100000 & 0.050000 & 0.010000 \\
num_qubits &  &  &  &  \\
\midrule
3 & 0.9065 & 0.9065 & 0.4725 & 0.5433 \\
4 & 0.8897 & 0.9134 & 0.1379 & 0.0051 \\
5 & 0.5007 & 0.5006 & 0.4035 & 0.3220 \\
6 & 0.5946 & 0.6132 & 0.5150 & 0.4065 \\
7 & 0.7732 & 0.7421 & 0.6964 & 0.6888 \\
8 & 0.7425 & 0.7425 & 0.7518 & 0.7060 \\
9 & 0.7612 & 0.7612 & 0.9935 & 0.9260 \\
10 & 0.9066 & 0.9031 & 0.7681 & 0.9551 \\
11 & 0.8134 & 0.8325 & 0.9192 & 0.8450 \\
12 & 0.9659 & 0.9638 & 0.8601 & 0.9646 \\
13 & 0.8974 & 0.9171 & 0.8709 & 0.9729 \\
14 & 0.9908 & 

In [17]:
df_mean_original < df_mean_modified

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,False,False,False,False
4,False,False,False,False
5,False,False,True,False
6,False,False,True,True
7,False,False,False,False
8,False,False,False,True
9,False,False,False,False
10,False,False,False,True
11,True,True,True,False
